In [ ]:
# https://www.kaggle.com/code/amlanmohanty1/face-recognition-attendance-system-using-dlib/notebook
# https://github.com/amlanmohanty1/Attendance-system-using-Face-Recognition

In [1]:
!pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached dlib-19.24.0.tar.gz (3.2 MB)
  Using cached face_recognition_models-0.3.0.tar.gz (100.1 MB)
  Running setup.py clean for dlib
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=7ae26924f74ce7c92af2dcf6e5e0c88ba7117ae0db55a0d14263d7158f22656e
  Stored in directory: c:\users\rajag\appdata\local\pip\cache\wheels\b4\4b\8f\751e99d45f089bdf366a7d3e5066db3c2b84a62e4377f534d7
Successfully built face-recognition-models
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\rajag\\AppData\\Local\\Temp\\pip-install-6s4bxawt\\dlib_6d922c7c9f0e4e9289f0fe4e54e7080a\\setup.py'"'"'; __file__='"'"'C:\\Users\\rajag\\AppData\\Local\\Temp\\pip-install-6s4bxawt\\dlib_6d922c7c9f0e4e9289f0fe4e54e7080a\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\rajag\AppData\Local\Temp\pip-wheel-k4mq_vgp'
       cwd: C:\Users\rajag\AppData\Local\Temp\pip-install-6s4bxawt\dlib_6d922c7c9f0e4e9289f0fe4e54e7080a\
  Complete output (8 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\python\dlib\__init__.py' not found (or not a regular file)
  running build_ext
  
  ERROR: CMake must be installed to build dlib
  
  ------------

In [2]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
path = '../input/training-images/Training images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

In [ ]:
def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [ ]:
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime("%m/%d/%Y,%H:%M:%S")
                f.writelines(f'\n{name},{dtString}')

In [ ]:
encodeListKnown = findEncodings(images)
print('Encoding Complete')

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    success, img = cap.read()

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()

            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    cv2.imshow('Webcam', img)
    cv2.waitKey(1)